In [6]:
from cpe_dict_loader import *
from registry_query import *
from cpe_similarity import *
from ml_importance import *
import json,random

In [2]:
software_inventory = get_installed_software()
cpe_list = load_cpe_processed_json()

In [3]:
test_set = [{"Vendor":"Google Corporation LTD","Name":"Chrome 0.1.38","Version":"1.54345.123"},
            {"Vendor":"Random Corp","Name":"Discord Chat and Message 0.0.4","Version":"0.5"},
            {"Vendor":"Microsoft Corporation","Name":"Microsoft Edge","Version":"80.0"},
            {"Vendor":"Spotify","Name":"Spotify for Windows","Version":"2"},
            {"Vendor":"Python Software Foundation","Name":"Python 2.7 (x64) Binary","Version":"2"}]

with open("test_cpe_set.json","r") as file:
    test_set = json.load(file)

In [9]:
all_matches = []

random.shuffle(software_inventory)

for software in software_inventory:
    best_match = {"CPE":None,"Similarity":0}

    if software.get("Vendor") and software.get("Name"):  
        name_importance = return_importance_vector(software.get("Name"))
        vendor_importance = return_importance_vector(software.get("Vendor"))

    else:
        continue

    for cpe in cpe_list:         
        similarity_score = return_similarity(cpe[1],software.get("Vendor"),software.get("Name"),software.get("Version"))
        importance_score = return_importance_weighted_similarity(cpe[1],software.get("Vendor"),software.get("Name"),vendor_importance,name_importance)

        total_score = 1 / (1 + math.exp(-(similarity_score + importance_score))) #use sigmoid function to normalise between 0 and 1

        if total_score > best_match["Similarity"] and cpe[1][2] == "a" and cpe[1][10] in ["windows","*"]:
            best_match["CPE"] = cpe[0]
            best_match["Similarity"] = total_score

    all_matches.append({"Software":software,"Best_match":best_match})

    if all_matches[-1]["Best_match"]["Similarity"] > 0.0: print(all_matches[-1])


{'Software': {'Vendor': 'Microsoft Corporation', 'Name': 'Windows SDK for Windows Store Apps DirectX x86 Remote', 'Version': '10.1.19041.685'}, 'Best_match': {'CPE': 'cpe:2.3:a:microsoft:directx:10.1:-:*:*:*:*:*:*', 'Similarity': 0.7243391568771036}}
{'Software': {'Vendor': 'Microsoft Corporation', 'Name': 'WinRT Intellisense PPI - en-us', 'Version': '10.1.22000.832'}, 'Best_match': {'CPE': 'cpe:2.3:a:microsoft:defender_for_endpoint_edr_sensor:10.8047.22439.1056:*:*:*:*:*:*:*', 'Similarity': 0.8845575383616279}}
{'Software': {'Vendor': 'Microsoft Corporation', 'Name': 'Universal CRT Extension SDK', 'Version': '10.1.19041.685'}, 'Best_match': {'CPE': 'cpe:2.3:a:microsoft:directx:10.1:-:*:*:*:*:*:*', 'Similarity': 0.4762920912615728}}
{'Software': {'Vendor': 'Electronic Arts, Inc.', 'Name': 'Titanfall™ 2', 'Version': '1.0.1.3'}, 'Best_match': {'CPE': 'cpe:2.3:a:libgit2:libgit2:1.0.1:*:*:*:*:*:*:*', 'Similarity': 0.3244896478894252}}
{'Software': {'Vendor': 'Microsoft Corporation', 'Name'

KeyboardInterrupt: 